In [265]:
import fiona
import rasterio
import geopandas as gpd
import json
from shapely.geometry import box
from shapely.geometry import shape, mapping
import pandas as pd
import numpy as np
from geopandas import clip
import scipy
import scipy.spatial
import datetime

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [266]:
boundaries_path = r'C:\GitHub\many-to-many-dijkstra\burundi\gadm36_BDI.gpkg'

In [267]:
boundaries = gpd.read_file(boundaries_path)

In [268]:
crs = 4326
resolution = 0.00833

df = pd.DataFrame(columns=['X', 'Y'])
min_x=float(boundaries.bounds['minx'])
min_y=float(boundaries.bounds['miny'])
max_x=float(boundaries.bounds['maxx'])
max_y = float(boundaries.bounds['maxy'])
# create one-dimensional arrays for x and y
lon = np.arange(min_x, max_x, resolution)
lat = np.arange(min_y, max_y, resolution)
lon, lat = np.meshgrid(lon, lat)
df['X'] = lon.reshape((np.prod(lon.shape),))
df['Y'] = lat.reshape((np.prod(lat.shape),))
geo_df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.X, df.Y), crs=crs)
geo_df = gpd.clip(geo_df, boundaries)
geo_df = geo_df.to_crs(3395)

geo_df.to_file('points.shp', driver = 'ESRI Shapefile')
points = fiona.open('points.shp')

DriverError: points.shp is not a directory.

In [ ]:
def rasterize(vector_layer, raster_base_layer, outpul_file=None, value=None,
              nodata=-9999, compression='NONE', dtype=rasterio.uint8,
              all_touched=True, save=False):
    vector_layer = vector_layer.rename(columns={'geometry': 'geom'})
    if value:
        dff = vector_layer[[value, 'geom']]
        shapes = ((g, v) for v, g in zip(dff[value].values, dff['geom'].values))
    else:
        shapes = ((g, 1) for g in vector_layer['geom'].values)

    with rasterio.open(raster_base_layer) as src:
        image = features.rasterize(
            shapes,
            out_shape=src.shape,
            transform=src.transform,
            all_touched=all_touched,
            fill=nodata)

        out_meta = src.meta

        out_meta.update({"driver": "GTiff",
                         "height": 0.00833,
                         "width": 0.00833,
                         "transform": src.transform,
                         'compress': compression,
                         'dtype': dtype,
                         "crs": src.crs,
                         'nodata': nodata})

        if save:
            with rasterio.open(outpul_file, 'w', **out_meta) as dst:
                dst.write(image, indexes=1)
        else:
            return image, out_meta

### Add roads network to cost layer

In [ ]:
roads_path = r'C:\GitHub\many-to-many-dijkstra\burundi\roads.gpkg'
roads = gpd.read_file(roads_path).to_crs(4326)

In [ ]:
def processing_lines(lines, admin, crs, workspace, points, name, gpd_points):
    print(datetime.datetime.now())
    
    lines_clip = clip(lines, admin)
    lines_clip.crs = {'init' :'epsg:4326'}
    lines_proj=lines_clip.to_crs({ 'init': crs})

    lines_proj.to_file(workspace + r"\ " + 'temp' + "_proj.shp", driver='ESRI Shapefile')

    line = fiona.open(workspace +  r"\ " + 'temp' + "_proj.shp")
    firstline = line.next()

    schema = {'geometry' : 'Point', 'properties' : {'id' : 'int'},}
    with fiona.open(workspace + r"\ " + 'temp' + "_proj_points.shp", "w", "ESRI Shapefile", schema) as output:
        for lines in line:
            if lines["geometry"] is not None:
                first = shape(lines['geometry'])
                length = first.length
                for distance in range(0,int(length),100):
                    point = first.interpolate(distance)
                    output.write({'geometry' :mapping(point), 'properties' : {'id':1}})

    lines_f = fiona.open(workspace + r"\ " + 'temp' + "_proj_points.shp")
    lines = gpd.read_file(workspace +  r"\ " + 'temp' + "_proj.shp")

    print(datetime.datetime.now())
    
    geoms1 = [shape(feat["geometry"]) for feat in lines_f]
    s1 = [np.array((geom.xy[0][0], geom.xy[1][0])) for geom in geoms1]
    s1_arr = np.array(s1)

    geoms2 = [shape(feat["geometry"]) for feat in points]
    s2 = [np.array((geom.xy[0][0], geom.xy[1][0])) for geom in geoms2]
    s2_arr = np.array(s2)
    
    print(datetime.datetime.now())

    def do_kdtree(combined_x_y_arrays,points):
        mytree = scipy.spatial.cKDTree(combined_x_y_arrays)
        dist, indexes = mytree.query(points)
        return dist, indexes

    print(datetime.datetime.now())
    
    dist, indexes = do_kdtree(s1_arr,s2_arr)

    z=dist.tolist()
    y=indexes.tolist()
    gpd_points[name+'Dist'] = z


    print(datetime.datetime.now())
    return gpd_points

In [ ]:
road_types = ['motorway', 'trunk', 'primary', 'secondary', 'tertiary', 'unclassified', 'residential', 'service']

In [ ]:
for road in road_types:
    selected_roads = roads.loc[roads['highway'] == road]
    geo_df = processing_lines(selected_roads, boundaries, 'EPSG:3395', 'burundi', points, road, geo_df)

In [ ]:
geo_df